In [6]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json
from requests_html import HTMLSession
import nest_asyncio
from dateutil import parser

from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

past_end_date = False

columns = ['date', 'press title', 'urls', '1d change', 'index_price','norm_price']

In [8]:
data = []

ticker = 'wmt'
sector = get_sector(ticker)

stock_hist = get_df(ticker)
index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)
page_num = 0
while True:
    
    nest_asyncio.apply()

    s = HTMLSession()
    url = f'https://corporate.walmart.com/news/?newsSearchModuleQuery=&0000017d-bf8a-d566-affd-bfba3416000c-page={page_num}'
    r = s.get(url)
    
    soup = BeautifulSoup(r.text) 


    press_statements = soup.find_all('div', {"class": "PromoCardNews"})
    
    if not press_statements:
        break
        
    for release in press_statements:
        date = release.find('date', {"class": "Promo-date"}).text.replace('.', '')
        title = release.find('h3', {"class": "PromoCardNews-title"}).text.lstrip().strip()
        url = release.find('a').get('href')
    
        
        category = release.find('span', {'class': 'PromoCardNews-eyebrow'})
        if category == None:
            continue
        else:
            category = category.text
            
        if category != 'Finance':
            continue        
        content = get_content_header(url, 'meta', {'name':'twitter:description'})
        date = parser.parse(date).date()
        if date < datetime(2014, 1, 1).date():
            past_end_date = True
            break
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([date, title, url, content,pct_change, index_price, norm_price])
    if past_end_date: break
                
    page_num = page_num + 1

save_data(data, ticker, sector)

In [22]:
press_statements

[]